In [ ]:
%matplotlib inline

In [ ]:
import torch

# Session 2: Reverse mode differentiation and operator overloading

## Motivation

<div class="alert alert-block alert-danger">
<b>TODO: motivation on backprop; think about learning outcomes</b>
</div>

## Learning objectives

In today's session we will:
* Learn about *reverse mode* and the *operator overloading* approach, comparing them with forward mode and source transformation, respectively.
* Verify the consistency of code generated by Tapenade under forward and reverse mode using the *dot product test*.
* Try out the *autograd* operator overloading AD tool underpinnning PyTorch (as well as libtorch (C++) and FTorch (Fortran)).
* Learn about *checkpointing* and using AD to compute higher order derivatives.
* See a showcase of some more advanced use cases of AD using Firedrake.

## Preparations

#### Terminology

**Recall from session 1**: This course introduces the concept of *differentiable programming*, a.k.a. *automatic differentiation (AD)*, or *algorithmic differentiation*. We will use the acronym AD henceforth.

#### Notation

**Recall from session 1**: For a differentiable *mathematical* function $f:A\rightarrow\mathbb{R}$ with scalar input (i.e., a single value) from $A\subseteq\mathbb{R}$, we make use of both the Lagrange notation $f'(x)$ and Leibniz notation $\frac{\mathrm{d}f}{\mathrm{d}x}$ for its derivative.
We **do not** use the physics notation for derivatives, so if you ever see (e.g.) $\dot{x}$ then this is just a variable name, not the derivative of $x$.

**Recall from session 1**: When it comes to *forward* derivatives in code, we use the `_d` notation, which is standard in the AD literature.

When it comes to *reverse* mode derivatives in code, we use the `_b` notation (for "backward").

<div class="alert alert-block alert-danger">
<b>TODO: Where did the history section go?</b>
</div>

## Recall: directional derivative, a.k.a. Jacobian-vector product (JVP)

> Consider a vector-valued function $\mathbf{f}$ mapping from a subspace $A\subseteq\mathbb{R}^n$ into $\mathbb{R}^m$, for some $m,n\in\mathbb{N}$:
> $$\mathbf{f}:A\rightarrow\mathbb R^m.$$
>
> Given input $\mathbf{x}\in A$ and a *seed vector* $\dot{\mathbf{x}}\in\mathbb{R}^n$, forward mode AD allows us to compute the *action* (matrix-vector product)
$$\text{JVP}(\mathbf{f},\mathbf{x},\dot{\mathbf{x}}):=\nabla\mathbf{f}(\mathbf{x})\,\dot{\mathbf{x}}.$$
Again, think of the seed vector as being an input from outside of the part of the program being differentiated.
>
> Here $\nabla\mathbf{f}$ is referred to as the *Jacobian* for the map, so the above is known as a *Jacobian-vector product (JVP)*.

<div class="alert alert-block alert-warning">
<b>Note</b>
The computation is <em>matrix-free</em>. We don't actually need the Jacobian when we compute this product.
</div>

## Jacobian-transpose-vector product (JTVP)

Consider a vector-valued function $\mathbf{f}$ mapping from a subspace $A\subseteq\mathbb{R}^n$ into $\mathbb{R}^m$, for some $m,n\in\mathbb{N}$:
$$\mathbf{f}:A\rightarrow\mathbb{R}^m.$$

Given $\mathbf{x}\in A$ and a seed vector $\bar{\mathbf{y}}\in\mathbb{R}^m$, reverse mode AD allows us to compute the *transpose action* (transposed matrix-vector product)
$$\text{JTVP}(\mathbf{f},\mathbf{x},\bar{\mathbf{y}}):=\nabla\mathbf{f}(\mathbf{x})^T\bar{\mathbf{y}}.$$

<div class="alert alert-block alert-info">
<b>Optional exercise</b>

Convince yourself that the JTVP is well defined.

<b>Solution</b>

<details>
We have $\nabla\mathbf{f}(\mathbf{x})\in\mathbb{R}^{m\times n}$, so $\nabla\mathbf{f}(\mathbf{x})^T\in\mathbb{R}^{n\times m}$. Since $\bar{\mathbf{y}}\in\mathbb{R}^m$, the dimensions are appropriate to take the JTVP.
</details>
</div>

<div class="alert alert-block alert-warning">
<b>Note</b>
Again, the computation is <em>matrix-free</em>. We don't actually need the Jacobian or its transpose when we compute this product.
</div>

## Forward mode vs. reverse mode

For seed vectors $\dot{\mathbf{x}}\in\mathbb{R}^n$ and $\bar{\mathbf{y}}\in\mathbb{R}^m$, forward mode and reverse mode compute

$$
    \text{JVP}(\mathbf{f},\mathbf{x},\dot{\mathbf{x}}):=\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}
    \quad\text{and}\quad
    \text{JTVP}(\mathbf{f},\mathbf{x},\bar{\mathbf{y}}):=\nabla\mathbf{f}(\mathbf{x})^T\bar{\mathbf{y}}
$$
respectively.

* Forward mode is more appropriate if $n\ll m$, i.e., $\#inputs\ll\#outputs$.
  * e.g., sensitivity analysis or optimisation w.r.t. a small number of parameters.
* Reverse mode is more appropriate if $n\gg m$, i.e., $\#inputs\gg\#outputs$.
  * e.g., ODE/PDE-constrained optimisation (cost function), machine learning training (loss function), goal-oriented error estimation (quantity of interest).
* Forward mode is computed *eagerly*, whereas reverse mode is done separately from the primal run.
* Reverse mode tends to have higher memory requirements.

## $f$ \& $g$ example: reverse mode

Recall the $f$ \& $g$ example from session 1. The Fortran code for the two functions is copied in the repository at `session2/exercises/fg/f.f90` and `session2/exercises/fg/g.f90`.

```fortran
subroutine f(x, y, z)
  implicit none
  real, intent(in)  :: x, y
  real, intent(out) :: z
  z = x * y
end subroutine f
```

```fortran
subroutine g(z, v)
  implicit none
  real, intent(in)  :: z
  real, intent(out), dimension(2) :: v
  v = [sin(z), cos(z)]
end subroutine g
```

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
1. Either [install Tapenade](https://tapenade.gitlabpages.inria.fr/tapenade/distrib/README.html)* or visit the [Tapenade web interface](http://tapenade.inria.fr:8080/tapenade/index.jsp).
2. Run `tapenade -h` to review the options.
3. Apply Tapenade to each of these subroutines **in reverse mode**, which will compute the JTVP for some seed vector.
4. Inspect the output files `f_b.f90` and `g_b.f90` and check they are as you expect.
5. Inspect the message files `f_b.msg` and `g_b.msg`.

*Note that you will need to install Java if you don't already have it installed.

<b>Solutions 2</b>
    
<details>

```sh
$ tapenade -h
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
 Builds a differentiated program.
 Usage: tapenade [options]* filenames
  options:
   -head, -root <proc>     set the differentiation root procedure(s)
                           See FAQ for refined invocation syntax, e.g.
                           independent and dependent arguments, multiple heads...
   -tangent, -d            differentiate in forward/tangent mode (default)
   -reverse, -b            differentiate in reverse/adjoint mode
   -vector, -multi         turn on "vector" mode (i.e. multi-directional)
   -specializeactivity <unit_names or %all%>  Allow for several activity patterns per routine
   -primal, -p             turn off differentiation. Show pointer destinations
   -output, -o <file>      put all generated code into a single <file>
   -splitoutputfiles       split generated code, one file per top unit
   -outputdirectory, -O <directory>  put all generated files in <directory> (default: .)
   -I <includePath>        add a new search path for include files
   -tgtvarname <str>       set extension for tangent variables  (default %d)
   -tgtfuncname <str>      set extension for tangent procedures (default %_d)
   -tgtmodulename <str>    set extension for tangent modules and types (default %_diff)
   -adjvarname <str>       set extension for adjoint variables  (default %b)
   -adjfuncname <str>      set extension for adjoint procedures (default %_b)
   -adjmodulename <str>    set extension for adjoint modules and types (default %_diff)
   -modulename <str>       set extension for tangent&adjoint modules and types (default %_diff)
   -inputlanguage <lang>   language of  input files (fortran, fortran90,
                           fortran95, or C)
   -outputlanguage <lang>  language of output files (fortran, fortran90,
                           fortran95, or C)
   -ext <file>             incorporate external library description <file>
   -nolib                  don't load standard libraries descriptions
   -i<n>                   count <n> bytes for an integer (default -i4)
   -r<n>                   count <n> bytes for a real (default -r4)
   -dr<n>                  count <n> bytes for a double real (default -dr8)
   -p<n>                   count <n> bytes for a pointer (default -p8)
   -fixinterface           don't use activity to filter user-given (in)dependent vars
   -noinclude              inline include files
   -debugTGT               insert instructions for debugging tangent mode
   -debugADJ               insert instructions for debugging adjoint mode
   -tracelevel <n>         set the level of detail of trace milestones
   -msglevel <n>           set the level of detail of error messages
   -msginfile              insert error messages in output files
   -dump <file>            write a dump <file>
   -html                   display results in a web browser
   -nooptim <str>          turn off optimization <str> (in {activity, difftypes,
                           diffarguments, stripprimalmodules, spareinit, splitdiff, 
                           mergediff, saveonlyused, tbr, snapshot, diffliveness,
                           deadcontrol, recomputeintermediates,
                           everyoptim}
   -version                display Tapenade version information
 Report bugs to <tapenade@inria.fr>.
```

</details>

<br>
<b>Solutions 3 and 4</b>
    
<details>

Running
```sh
$ cd session2/exercises/fg
$ ls
f.f90 g.f90
$ tapenade -b f.f90 
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Command: Took subroutine f as default differentiation root
@@ Created ./f_b.f90 
@@ Created ./f_b.msg
$ cat f_b.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of f in reverse (adjoint) mode:
!   gradient     of useful results: x y z
!   with respect to varying inputs: x y z
!   RW status of diff variables: x:incr y:incr z:in-zero
SUBROUTINE F_B(x, xb, y, yb, z, zb)
  IMPLICIT NONE
  REAL, INTENT(IN) :: x, y
  REAL :: xb, yb
  REAL :: z
  REAL :: zb
  xb = xb + y*zb
  yb = yb + x*zb
  zb = 0.0
END SUBROUTINE F_B
```

Running
```sh
$ tapenade tapenade -b g.f90 
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
Command: Took subroutine g as default differentiation root
@@ Created ./g_b.f90 
@@ Created ./g_b.msg
$ cat g_b.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of g in reverse (adjoint) mode:
!   gradient     of useful results: v z
!   with respect to varying inputs: v z
!   RW status of diff variables: v:in-zero z:incr
SUBROUTINE G_B(z, zb, v, vb)
  IMPLICIT NONE
  REAL, INTENT(IN) :: z
  REAL :: zb
  REAL, DIMENSION(2) :: v
  REAL, DIMENSION(2) :: vb
  INTRINSIC COS
  INTRINSIC SIN
  zb = zb + COS(z)*vb(1) - SIN(z)*vb(2)
  vb = 0.0
END SUBROUTINE G_B
```

</details>
</div>

## Verification: the dot product test

We have approaches for computing derivatives with both forward mode and reverse mode. But how do we know the outputs are consistent? This can be verified using the so-called *dot product test*.

In the dot product test, we define a quantity $\psi:=\bar{\mathbf{y}}\dot{\mathbf{y}}$, where $\dot{\mathbf{y}}=\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}$ is the output of forward mode and check that $\psi=\bar{\mathbf{x}}\dot{\mathbf{x}}$, where $\bar{\mathbf{x}}^T=\nabla\mathbf{f}(\mathbf{x})^T\bar{\mathbf{y}}$ is the output of reverse mode.

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
Prove that we do expect $\psi=\bar{\mathbf{x}}\dot{\mathbf{x}}$.

<b>Solution</b>
    
<details>

$$
    \psi:=\bar{\mathbf{y}}\dot{\mathbf{y}}
    =\bar{\mathbf{y}}(\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}})
    =(\bar{\mathbf{y}}\nabla\mathbf{f}(\mathbf{x}))\dot{\mathbf{x}}
    =\bar{\mathbf{x}}\dot{\mathbf{x}}
$$

</details>

</div>

## $f$ \& $g$ example: dot product test

We've already computed forward mode and reverse mode derivatives for the function $f$ using Tapenade. We can run a dot product test using the code in `session2/exercises/fg/dot_product_test.f90`.

```fortran
include "f_d.f90"
include "f_b.f90"

! Program for verifying the consistency of the forward mode and reverse mode derivatives of the
! function f.
program dot_product_test
  implicit none

  real :: x, xd, xb
  real :: y, yd, yb
  real :: z, zd, zb

  real :: result1, result2
  real, parameter :: atol = 1e-05

  ! Set arbitrary primal input
  x = 1.2
  y = -2.3

  ! Call forward mode with some arbitrary seeds
  xd = 4.2
  yd = -0.7
  call f_d(x, xd, y, yd, z, zd)

  ! Choose a seed for reverse mode and evaluate the first result
  zb = 3.0
  result1 = zd * zb

  ! Call reverse mode and evaluate the second result
  xb = 0.0
  yb = 0.0
  call f_b(x, xb, y, yb, z, zb)
  result2 = xd * xb + yd * yb

  ! Check the two results match within the prespecified tolerance
  if (abs(result1 - result2) < atol) then
    write(unit=6, fmt="('PASS')")
  else
    write(unit=6, fmt="('FAIL with atol=',e10.4)") atol
  end if

end program dot_product_test
```

<div class="alert alert-block alert-info">
<b>Exercise</b>
    
1. Build and run the `dot_product_test` program.
2. Why does the test fail if the `result1` assignment is moved to after the call to `f_b`?

<b>Solution 1</b>
    
<details>

```sh
$ cd exercises/fg
$ gfortran dot_product_test.f90 -o dot_product_test
$ ./dot_product_test
PASS
```

</details>

<br>
<b>Solution 2</b>
    
<details>

Because `zb` gets reset to zero by `f_b`. This is done by default because it's almost always the Right Thing to do.

</details>

</div>

## Second order

For seed vectors $\dot{\mathbf{x}}\in\mathbb{R}^n$ and $\bar{\mathbf{y}}\in\mathbb{R}^m$, forward mode and reverse mode compute

$$
    \text{JVP}(\mathbf{f},\mathbf{x},\dot{\mathbf{x}}):=\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}
    \quad\text{and}\quad
    \text{JTVP}(\mathbf{f},\mathbf{x},\bar{\mathbf{y}}):=\nabla\mathbf{f}(\mathbf{x})^T\bar{\mathbf{y}}
$$
respectively.

<div class="alert alert-block alert-info">
<b>Question</b>
    
What are two ways we can we use these to compute the Hessian of $f$?

<b>Solution 1</b>
    
<details>

Given a seed vector $\dot{\mathbf{x}}$, first apply forward mode to compute $\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}$. Then apply forward mode to compute the gradient of *this* (i.e., apply forward mode to the *forward mode derivative code*). Use vector mode (preferably with compression!) to get the full Hessian.

</details>

<br>
<b>Solution 2</b>
    
<details>

Given a seed vector $\dot{\mathbf{x}}$, first apply forward mode to compute $\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}$. Then apply reverse mode to compute the gradient of *this* (i.e., apply reverse mode to the *forward mode derivative code*). That is, $(\nabla(\nabla\mathbf{f}(\mathbf{x})\dot{\mathbf{x}}))^T\bar{\mathbf{y}}=\dot{\mathbf{x}}^T\nabla^T\nabla\mathbf{f}(\mathbf{x})\bar{\mathbf{y}}$. Here the Hessian $\mathbf{H}(\mathbf{f}):=\nabla^T\nabla\mathbf{f}(\mathbf{x})$ is symmetric and so the two applications give the Hessian-vector product with the seed. Use vector mode (preferably with compression!) to get the full Hessian.

</details>

</div>

## Speelpenning example

We compute the Hessian for the classic test case introduced by Speelpenning, which amounts to a reduction of a vector using the product:
$$
    f(\mathbf{x})=\prod_{i=1}^nx_i.
$$
This test case is of interest because its Hessian is dense.

The Speelpenning function is implemented as a Fortran subroutine in `session2/exercises/speelpenning/speelpenning.f90`:
```fortran
subroutine speelpenning(x, f, n)
  implicit none
  integer, intent(in) :: n
  real, dimension(n), intent(in) :: x
  real, intent(out) :: f
  integer :: i
  f = 1.0
  do i = 1, n
    f = f * x(i)
  end do
end subroutine speelpenning
```

<div class="alert alert-block alert-info">
<b>Exercises</b>

1. Compute the Hessian of the `speelpenning` subroutine using Tapenade.*
2. Build and run the `view_hessian` program in `session2/exercises/speelpenning/view_hessian.f90` and convince yourself that the output is as you expect.

*Hint: One of the two approaches is much easier than the other!

<b>Solution 1</b>
    
<details>

```sh
$ tapenade -vector speelpenning.f90
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
@@ Options:  multiDirectional
Command: Took subroutine speelpenning as default differentiation root
Diff-liveness analysis turned off
@@ Created ./speelpenning_dv.f90 
@@ Created ./speelpenning_dv.msg
$ cat speelpenning_dv.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of speelpenning in forward (tangent) mode (with options multiDirectional):
!   variations   of useful results: f
!   with respect to varying inputs: x
!   RW status of diff variables: f:out x:in
SUBROUTINE SPEELPENNING_DV(x, xd, f, fd, n, nbdirs)
  USE DIFFSIZES
!  Hint: nbdirsmax should be the maximum number of differentiation directions
  IMPLICIT NONE
  INTEGER, INTENT(IN) :: n
  REAL, DIMENSION(n), INTENT(IN) :: x
  REAL, DIMENSION(nbdirsmax, n), INTENT(IN) :: xd
  REAL, INTENT(OUT) :: f
  REAL, DIMENSION(nbdirsmax), INTENT(OUT) :: fd
  INTEGER :: i
  INTEGER :: nd
  INTEGER :: nbdirs
  f = 1.0
  fd = 0.0
  DO i=1,n
    DO nd=1,nbdirs
      fd(nd) = x(i)*fd(nd) + f*xd(nd, i)
    END DO
    f = f*x(i)
  END DO
END SUBROUTINE SPEELPENNING_DV
```
We need to define the `diffsizes` module. Create `diffsizes.f90` containing
```fortran
module diffsizes
  implicit none
  integer, parameter :: nbdirsmax = 7
end module diffsizes
```

Inline this with the VJP code:
```sh
$ cat diffsizes.f90 speelpenning_dv.f90 > tmp.f90
```

Then we can apply vector forward mode:
```sh
$ tapenade -vector -head "speelpenning_dv(fd)/(x)" tmp.f90 -o speelpenning_dv
Tapenade 3.16 (develop) - 23 Apr 2025 13:39 - Java 21.0.7 Linux
@@ TAPENADE_HOME=/home/joe/software/tools/tapenade_3.16/bin/..
@@ Options:  multiDirectional
Diff-liveness analysis turned off
@@ Created ./speelpenning_dv_dv.f90 
@@ Created ./speelpenning_dv_dv.msg
$ cat speelpenning_dv_dv.f90
```
gives
```fortran
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of speelpenning_dv in forward (tangent) mode (with options multiDirectional):
!   variations   of useful results: fd
!   with respect to varying inputs: x
!   RW status of diff variables: x:in fd:out
!        Generated by TAPENADE     (INRIA, Ecuador team)
!  Tapenade 3.16 (develop) - 23 Apr 2025 13:39
!
!  Differentiation of speelpenning in forward (tangent) mode (with options multiDirectional):
!   variations   of useful results: f
!   with respect to varying inputs: x
!   RW status of diff variables: f:out x:in
SUBROUTINE SPEELPENNING_DV_DV(x, xd0, xd, f, fd, fdd, n, nbdirs, nbdirs0&
&)
  USE DIFFSIZES
  USE DIFFSIZES
!  Hint: nbdirsmax0 should be the maximum number of differentiation directions
  IMPLICIT NONE
  INTEGER, INTENT(IN) :: n
  REAL, DIMENSION(n), INTENT(IN) :: x
  REAL, DIMENSION(nbdirsmax0, n), INTENT(IN) :: xd0
  REAL, DIMENSION(nbdirsmax, n), INTENT(IN) :: xd
  REAL, INTENT(OUT) :: f
  REAL, DIMENSION(nbdirsmax0) :: fd0
  REAL, DIMENSION(nbdirsmax), INTENT(OUT) :: fd
  REAL, DIMENSION(nbdirsmax0, nbdirsmax), INTENT(OUT) :: fdd
  INTEGER :: i
  INTEGER :: nd
  INTEGER :: nbdirs
  INTEGER :: nd0
  INTEGER :: nbdirs0
  f = 1.0
  fd = 0.0
  fd0 = 0.0
  fdd = 0.0
  DO i=1,n
    DO nd=1,nbdirs
      DO nd0=1,nbdirs0
        fdd(nd0, nd) = fd(nd)*xd0(nd0, i) + x(i)*fdd(nd0, nd) + xd(nd, i&
&         )*fd0(nd0)
      END DO
      fd(nd) = x(i)*fd(nd) + f*xd(nd, i)
    END DO
    DO nd0=1,nbdirs0
      fd0(nd0) = x(i)*fd0(nd0) + f*xd0(nd0, i)
    END DO
    f = f*x(i)
  END DO
END SUBROUTINE SPEELPENNING_DV_DV
```

</details>

<br>
<b>Solution 2</b>
    
<details>

Running
```sh
$ cd session2/exercises/speelpenning
$ gfortran view_hessian.f90 -o view_hessian
$ ./view_hessian
```
should give the output
```default
   0.0 2520.0 1680.0 1260.0 1008.0  840.0  720.0
2520.0    0.0  840.0  630.0  504.0  420.0  360.0
1680.0  840.0    0.0  420.0  336.0  280.0  240.0
1260.0  630.0  420.0    0.0  252.0  210.0  180.0
1008.0  504.0  336.0  252.0    0.0  168.0  144.0
 840.0  420.0  280.0  210.0  168.0    0.0  120.0
 720.0  360.0  240.0  180.0  144.0  120.0    0.0
```
It makes sense that the diagonal entries are zero. The matrix is symmetric, as expected. The $(6,7)^{th}$ and $(7,6)^{th}$ entries are both 
$$120=5!=\prod_{i=1}^5x_i=\frac{\partial^2}{\partial x_6\partial x_7}\prod_{i=1}^7x_i.$$
Further checks left as an exercise to the reader.

</details>

</div>

## Approach 2: Operator overloading

<div class="alert alert-block alert-danger">
<b>TODO: gradient values, tape, overloaded operators, tape unrolling</b>
</div>

#### Operator overloading tools:

* LLVM
    * [Enzyme](https://enzyme.mit.edu) <!-- is a plugin that performs automatic differentiation (AD) of statically analyzable LLVM. By operating on the LLVM level Enzyme is able to perform AD across a variety of languages (C/C++, Fortran , Julia, etc.) and perform optimization prior to AD -->
* C/C++
    * About 2 dozen AD tools!
    * e.g., [ADIC](https://www.mcs.anl.gov/research/projects/adic), [ADOL-C](https://github.com/coin-or/ADOL-C), [Torch Autograd](https://pytorch.org/tutorials/advanced/cpp_autograd.html), [CoDiPack](https://github.com/SciCompKL/CoDiPack), [Sacado](https://docs.trilinos.org/dev/packages/sacado/doc/html/index.html), [dco/c++](https://nag.com/automatic-differentiation) [commercial]
* Fortran
    * [Differentia](https://github.com/Nicholaswogan/Differentia), [lots of abandonware...]
* Python
    * [PyADOL-C](https://github.com/b45ch1/pyadolc), [Jax](https://github.com/jax-ml/jax), [PyTorch Autograd](https://pytorch.org/docs/stable/autograd.html)
* Julia
    * About 2 dozen AD tools! https://juliadiff.org/
    * e.g., Enzyme, [Zygote](https://fluxml.ai/Zygote.jl/stable), [ForwardDiff](https://juliadiff.org/ForwardDiff.jl/stable)
    * [DifferentiationInterface](https://www.juliapackages.com/p/differentiationinterface)
* Domain-specific
    * [dolfin-adjoint/pyadjoint](https://github.com/dolfin-adjoint/pyadjoint) (Python/UFL - Firedrake & FEniCS)
* And many more! https://autodiff.org/?module=Tools

## PyTorch reverse mode demo

Builds upon the PyTorch Autograd tutorial: https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html.

Consider two tensors comprised of vectors of length 2. We pass `requires_grad=True` to the constructor to mark these tensors as active for differentiation.

In [ ]:
a = torch.tensor([2.0, 3.0], requires_grad=True)
b = torch.tensor([6.0, 4.0], requires_grad=True)

Construct another tensor as a mathematical combination of the first two tensors.

In [ ]:
Q = 3 * (a ** 3 - b * b / 3)
print(f"Q = {Q}")

<div class="alert alert-block alert-danger">
<b>TODO: Remark on `grad_fn`</b>
</div>

Next, we compute derivatives of `Q` with its `backward` method. The backward method accepts an optional argument for an external gradient. This defaults to a tensor of ones of the appropriate shape, but can be customised to account for cases where gradient values are propagated from another model or subcomponent.

In [ ]:
external_gradient = torch.ones((2,))
Q.backward(gradient=external_gradient)

We call the `backward` method on tensor `Q` to compute gradients with respect to any of its dependencies that were constructed with the `requires_grad=True` setting. As such, we may compute $\mathrm{d}Q/\mathrm{d}a$ and $\mathrm{d}Q/\mathrm{d}b$ as follows:

In [ ]:
print(f"a = {a.grad}")
print(f"b = {b.grad}")

#### Exercise 1

Convince yourself that these gradient values are correct. That is, differentiate the expression
$$Q=3(a^3-b^2/3)$$
with respect to $a$, substitute the values $a=(2,3)$ and $b=(6,4)$, and check the values match those above. Then do the same thing differentiating with respect to $b$.

##  Mathematical background on adjoint methods

<div class="alert alert-block alert-danger">
<b>TODO: derivation; continuous vs. discrete </b>
</div>

## Checkpointing

<div class="alert alert-block alert-danger">
<b>TODO: overview </b>
</div>

## Exercise: Sensitivity analysis

<div class="alert alert-block alert-danger">
<b>TODO </b>
</div>

## Neural network training with backpropagation

<div class="alert alert-block alert-danger">
<b>TODO: torch example </b>
</div>

## Further applications

* Data assimilation. <!-- Uses gradients of cost functions involving mismatches against observations to assimilate the observations. -->
* Uncertainty quantification. <!-- Uses Hessians -->
* Online training in machine learning. <!-- Requires derivatives of code downstream from machine learning outputs -->
* PDE-constrained optimisation.  <!-- Typically uses gradient-based methods -->
* Goal-oriented error estimation and mesh adaptation. <!-- Uses adjoint solutions -->

## Levels of abstraction

So far, we have considered AD applied to elementary operations (`+`, `-`, `*`, `/`, `**`) in Fortran. Most of the early AD tools worked in this way, tracking operations applied to real numbers and chaining together their derivatives. However, code differentiated with a source transformation tool in this manner can quickly become difficult to read for large codes. And under the operator overloading approach, the memory footprint of the tape can become very large.

By raising the level of abstraction, some of these difficulties can be avoided.

#### Medium-level: API calls

Examples:
* AD in PETSc
  * https://github.com/SciCompKL/adjoint-PETSc.
  * (Note: [old draft implementation](https://gitlab.com/petsc/petsc/-/tree/main/src/ts/tutorials/autodiff) differentiated elementary operators.)

#### High-level: operations on fields

Examples:
* AD in Firedrake using Pyadjoint/dolfin-adjoint
  * https://nbviewer.org/github/firedrakeproject/firedrake/blob/master/docs/notebooks/06-pde-constrained-optimisation.ipynb.
  * https://nbviewer.org/github/firedrakeproject/firedrake/blob/master/docs/notebooks/11-extract-adjoint-solutions.ipynb.
* AD in PSyclone using PSyAD.
  * https://github.com/stfc/PSyclone/tree/master/examples/psyad.

## PDE-constrained optimisation showcase

<div class="alert alert-block alert-danger">
<b>TODO: Firedrake </b>
</div>

## Goal-oriented mesh adaptation

<div class="alert alert-block alert-danger">
<b>TODO: Firedrake </b>
</div>

## Summary

In today's session we:
* Learnt about *reverse mode* and the *operator overloading* approach, comparing them with forward mode and source transformation, respectively.
* Verified the consistency of code generated by Tapenade under forward and reverse mode using the *dot product test*.
* Tried out the *autograd* operator overloading AD tool underpinnning PyTorch (as well as libtorch (C++) and FTorch (Fortran)).
* Learnt about *checkpointing* and using AD to compute higher order derivatives.
* Saw a showcase of some more advanced use cases of AD using Firedrake.

## References

* S. Linnainmaa. *Taylor expansion of the accumulated rounding error*. BIT,
16(2):146–160, 1976.
* B. Speelpenning. *Compiling fast partial derivatives of functions given by algorithms*.
University of Illinois, 1980.
* A. Griewank. *Achieving logarithmic growth of temporal and spatial complexity in
reverse automatic differentiation.* Optimization Methods & Software, 1:35–54, 1992.
* J. Huckelheim, et al. *A taxonomy of automatic differentiation pitfalls*, WIREs Data Mining Knowl Discovery, 2024.